# Классификация комментариев

**Задача** - построить модель классификации текстов со значением метрики качества *F1* >= 0.75. 

**Ключевые шаги**  
1. Обзор данных
2. Подготовка текстовой переменной 
3. Обучение моделей

**Описание**  
Интернет-магазин запускает новый сервис, где пользователи могут редактировать и дополнять описания товаров. Клиенты смогут предлагать правки и комментировать изменения других. Необходимо разработать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется разработать модель, которая будет определять комментарий позитивный или негативный. Предоставлены размеченные данные с пометкой о токсичности и тексты комментариев.

**Библиотеки**  
pandas, sklearn, nltk, pytorch, catboost, lightgbm, matplotlib 

### Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install transformers
!pip install torch
!pip install nltk

     |████████████████████████████████| 2.3MB 29.9MB/s 
     |████████████████████████████████| 901kB 34.9MB/s 
     |████████████████████████████████| 3.3MB 32.4MB/s 


In [3]:
import torch
import transformers
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import notebook

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
!pip install pandarallel
from pandarallel import pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.2-cp37-none-any.whl size=18386 sha256=393de5ceb9ce9f98f7b84fa1265f1ceca4d9e194b0db8c45cba6761f7769b9a0
  Stored in directory: /root/.cache/pip/wheels/40/80/6d/d50fb72a8ce6a923fb10390fec9eaaa40b02d07a7ec05c9c05
Successfully built pandarallel


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score

In [6]:
!pip install catboost

     |████████████████████████████████| 67.3MB 62kB/s 


In [7]:
!pip install lightgbm

In [8]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [9]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('max_columns', None)

In [11]:
df = pd.read_csv('drive/My Drive/datasets/toxic_comments.csv')

#### Обзор данных

In [12]:
df.sample(5)

,text,toxic
107690,REDIRECT Talk:Second Time Around (TV series),0
29602,"""\n\n Manfred I Lancia \n\nMy apologies, but I...",0
58513,Really distastefull opinonated personnal attac...,0
27695,I've already restored the separated articles. ...,0
104893,It would be inappropriate to split this articl...,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [14]:
df['toxic'].value_counts(normalize=True)

0   0.90
1   0.10
Name: toxic, dtype: float64

Нам предоставлены данные с описаниями товаров от посетителей сайта и оценкой их токсичности. Всего в датафрейме содержатся 159,6 тыс. описаний. В 90% случаев они отражают отрицательный смысл

Уберём знакми препинания и лишние символы

In [15]:
df['clear_text'] = df['text'].str.replace('[^a-zA-Z]', ' ')

In [16]:
stopwords = set(nltk_stopwords.words('english'))

In [17]:
def preclean(text):
    token_text = word_tokenize(text.lower())
    wo_sw = ' '.join([word for word in token_text if not word in stopwords])
    return wo_sw

In [18]:
%%time
pandarallel.initialize(verbose=1)
df['clear_text'] = df['clear_text'].parallel_apply(preclean)

CPU times: user 489 ms, sys: 297 ms, total: 786 ms
Wall time: 46.1 s


In [19]:
df.sample(5)

,text,toxic,clear_text
118051,Really? You can't see the blown highlights in...,0,really see blown highlights shirt paleness face
132962,I just wanted to add that there is a differenc...,0,wanted add difference may th movement may th i...
137012,"You don't add the info to an article, simply t...",0,add info article simply discount go realize di...
13018,The hypocricy of Administrators \n\nThere is n...,1,hypocricy administrators community community f...
109389,Go to hell \n\nFuck you,1,go hell fuck


### Очищение текста и лемматизация

In [20]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.VERB)

In [21]:
def lemmatizer(text):
    m = WordNetLemmatizer()
    lemm = ' '.join([m.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    return lemm

In [22]:
%%time
pandarallel.initialize(verbose=1)
df['lemm_text'] = df['clear_text'].parallel_apply(lemmatizer) 

CPU times: user 3.88 s, sys: 723 ms, total: 4.6 s
Wall time: 12min 53s


In [23]:
df.sample(5)

,text,toxic,clear_text,lemm_text
109385,"And that was why I restored it those times, an...",0,restored times removed tag thumperwad consider...,restore time remove tag thumperwad considers t...
147756,September\n Please stop. If you continue to va...,0,september please stop continue vandalize pages...,september please stop continue vandalize page ...
131207,"""\n\n Where? \n\nI surrender. Non-attorneys ha...",0,surrender non attorneys decided entry genuine ...,surrender non attorney decide entry genuine co...
75407,"""\n I agree, someone should change this map, ...",0,agree someone change map clearly states soviet...,agree someone change map clearly state soviet ...
145671,"Edit request from Ncmahesh, 14 October 2010 \n...",0,edit request ncmahesh october baton design cap...,edit request ncmahesh october baton design cap...


#### TF-IDF

In [24]:
tf_idf = TfidfVectorizer()

## Обучение

In [25]:
Y = df['toxic']

In [26]:
X = df['lemm_text']

In [27]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.25, random_state=42)

In [28]:
def result(prediction):
    print('F1: {:.3f}'.format(f1_score(Y_valid, prediction)))
    print(classification_report(Y_valid, prediction))
    print('AUC: {:.3f}'.format(roc_auc_score(Y_valid, prediction)))

In [29]:
train = tf_idf.fit_transform(X_train)

In [30]:
valid = tf_idf.transform(X_valid)

### Логистическая регрессия

In [31]:
%%time
lr = LogisticRegression(random_state=42, class_weight='balanced')
lr.fit(train, Y_train)

CPU times: user 8.47 s, sys: 9.65 s, total: 18.1 s
Wall time: 9.36 s


In [32]:
pred_lr = lr.predict(valid)

In [33]:
result(pred_lr)

F1: 0.751
              precision    recall  f1-score   support

           0       0.98      0.95      0.97     35834
           1       0.67      0.86      0.75      4059

    accuracy                           0.94     39893
   macro avg       0.83      0.90      0.86     39893
weighted avg       0.95      0.94      0.95     39893

AUC: 0.905


In [34]:
lr_sc = f1_score(Y_valid, pred_lr).round(3)
lr_auc = roc_auc_score(Y_valid, pred_lr).round(3)

Логистическая регрессия с учётом балансирования данных, показала целевой F1 = 0.75. Доля правильно предсказанных ответов - 0.94, доля правильно предсказанных среди всех предсказанных = 0.86. Точность предсказания показала, что довольно много объектов отрицательного класса были предсказаны как положительные (ошибки I рода). ROC-AUC = 0.91.  
Время обучения и предсказания является преимуществом данной модели.

### Случайный лес

In [35]:
parameters_rf = { 'n_estimators': [20],
                  'class_weight': ['balanced'],
                  'random_state': [42]}

In [36]:
rf = RandomForestClassifier()
grid_rf = GridSearchCV(rf, parameters_rf, scoring = 'f1', cv=2, n_jobs=-1)

In [37]:
%%time
grid_rf.fit(train, Y_train)

CPU times: user 1min 53s, sys: 175 ms, total: 1min 53s
Wall time: 3min 49s


GridSearchCV(cv=2, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [38]:
pred_rf = grid_rf.predict(valid)
result(pred_rf)

F1: 0.616
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     35834
           1       0.94      0.46      0.62      4059

    accuracy                           0.94     39893
   macro avg       0.94      0.73      0.79     39893
weighted avg       0.94      0.94      0.93     39893

AUC: 0.728


In [39]:
rf_sc = f1_score(Y_valid, pred_rf).round(3)
rf_auc = roc_auc_score(Y_valid, pred_rf).round(3)

Модель случайного леса с параметром кол-ва деревьев=20 и сбалансированной выборкой показала F1 = 0.62, что заметно уступает другим моделям. Модель лучше предсказывает нулевой класс и определила меньше половины действительно токсичных комментариев из всех в выборке.
Также можно отметить, что модель требует большей оперативности процессора при увеличении числа деревьев, что можно отнести к недостаткам.  

### LGBM

In [40]:
%%time
lgbm = LGBMClassifier(n_estimators = 1000, eval_metric = 'F1',  random_state=42)
lgbm.fit(train, Y_train, eval_set=(valid, Y_valid), verbose=1000)
pred_lgbm = lgbm.predict(valid)

[1000]	valid_0's binary_logloss: 0.114964
CPU times: user 11min 15s, sys: 905 ms, total: 11min 16s
Wall time: 11min 13s


In [41]:
result(pred_lgbm)

F1: 0.788
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     35834
           1       0.88      0.71      0.79      4059

    accuracy                           0.96     39893
   macro avg       0.92      0.85      0.88     39893
weighted avg       0.96      0.96      0.96     39893

AUC: 0.851


In [42]:
lgbm_sc = f1_score(Y_valid, pred_lgbm).round(3)
lgbm_auc = roc_auc_score(Y_valid, pred_lgbm).round(3)

На основе сравнения показателей F1 модель lightgbm оказалась лучшей. Полнота предсказания заметно выше, чем у случайного леса, но хуже, чем у логита. Время обучения значительно выше небустинговых моделей.

### CatBoost

In [43]:
train = df[['toxic', 'lemm_text']].sample(frac=0.75, random_state=42).copy()
val = df[['toxic', 'lemm_text']][~df.index.isin(train.index)].copy()

In [44]:
XX = ['lemm_text']
YY = ['toxic']
text = ['lemm_text']

In [45]:
params = {'iterations': 200,
          'max_depth': 10,
          'auto_class_weights': 'Balanced',
          'eval_metric': 'F1',
          'text_features': text,
          'verbose': 100,
          'random_state': 42}

In [46]:
cat = CatBoostClassifier(**params)

In [47]:
%%time
cat.fit(train[XX], train[YY], eval_set = (val[XX], val[YY]))

Learning rate set to 0.20808
0:	learn: 0.8680243	test: 0.8806174	best: 0.8806174 (0)	total: 3.65s	remaining: 12m 6s
100:	learn: 0.9146824	test: 0.9038710	best: 0.9046358 (93)	total: 5m 58s	remaining: 5m 51s
199:	learn: 0.9334451	test: 0.9031306	best: 0.9049405 (145)	total: 11m 53s	remaining: 0us

bestTest = 0.904940538
bestIteration = 145

Shrink model to first 146 iterations.
CPU times: user 23min 26s, sys: 2.34 s, total: 23min 28s
Wall time: 12min


In [48]:
pred_cat = cat.predict(val[XX])
print('F1: {:.3f}'.format(f1_score(val[YY], pred_cat)))

F1: 0.724


In [49]:
print(classification_report(val[YY], pred_cat))

              precision    recall  f1-score   support

           0       0.99      0.94      0.96     35772
           1       0.62      0.88      0.72      4121

    accuracy                           0.93     39893
   macro avg       0.80      0.91      0.84     39893
weighted avg       0.95      0.93      0.94     39893



F1 = 0.72 меньше целевого значения, однако полнота предсказаний 1-го класса модели выше всех и также выше ROC-AUC. Если требуется найти как можно больше токсичных комментариев, то лучше использовать catboost, однако если требуется не перегрузить модерацию ложными токсичными текстами, то это не лучший выбор.

In [50]:
cat_sc = f1_score(val[YY], pred_cat).round(3)
cat_auc = roc_auc_score(val[YY], pred_cat).round(3)

## Выводы

In [51]:
data = {'Logit': [lr_sc, lr_auc],
        'RF': [rf_sc, rf_auc],
        'LGBM': [lgbm_sc, lgbm_auc],
        'CatB': [cat_sc, cat_auc] }

In [52]:
f1_scores = pd.DataFrame (data, columns = ['Logit','RF', 'LGBM', 'CatB'], index=['F1', 'AUC'])

In [53]:
f1_scores

,Logit,RF,LGBM,CatB
F1,0.75,0.62,0.79,0.72
AUC,0.91,0.73,0.85,0.91


**Вывод**  
В условиях задачи лучшей оказалась модель lightgbm с метрикой = 0.79, а хуже всего себя показал случайный лес. Однако если выбирать модель лучшую по времени обучения и с F1 >= 0.75, то лучше выбрать логистическую регрессию, которая также находит больше действительно токсичных комментариев.